In [3]:
!pip install dlib

In [1]:
pip install dlib-19.24.1-cp311-cp311-win_amd64.whl

Processing c:\users\soura\sec\dlib-19.24.1-cp311-cp311-win_amd64.whl
dlib is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip list

Package                                  Version
---------------------------------------- ---------------
absl-py                                  2.1.0
accelerate                               0.28.0
aiobotocore                              2.5.0
aiofiles                                 22.1.0
aiohttp                                  3.8.5
aioitertools                             0.7.1
aiosignal                                1.2.0
aiosqlite                                0.18.0
alabaster                                0.7.12
alembic                                  1.13.1
altair                                   5.2.0
anaconda-anon-usage                      0.4.2
anaconda-catalogs                        0.2.0
anaconda-client                          1.12.1
anaconda-cloud-auth                      0.1.3
anaconda-navigator                       2.5.2
anaconda-project                         0.11.1
annotated-types                          0.6.0
anyio                                    

In [3]:
!pip install cvzone

  Using cached cvzone-1.6.1-py3-none-any.whl


In [2]:
!pip install face-recognition

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)


# Encoding 

In [ ]:
# Initialize Firebase
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': "https://facial-recognition-623bd-default-rtdb.firebaseio.com/",
    'storageBucket': "facial-recognition-623bd.appspot.com"  
})

In [1]:
import os
import pickle
import numpy as np
import cv2
import face_recognition

# Path to images and encoding file
images_folder = 'img'
encoding_file = 'EncodeFile.p'

def encode_images(images_folder):
    """
    Encodes faces from images in the specified folder and saves encodings to a file.
    """
    encode_list = []
    student_ids = []
    
    for image_name in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_name)
        print(f"Processing image: {image_path}")
        
        img = cv2.imread(image_path)
        if img is None:
            print(f"Failed to load image: {image_path}")
            continue
        
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = face_recognition.face_locations(img_rgb)
        
        if not faces:
            print(f"No faces found in image: {image_path}")
            continue
        
        encodings = face_recognition.face_encodings(img_rgb, faces)
        if not encodings:
            print(f"Failed to encode image: {image_path}")
            continue
        
        encode_list.extend(encodings)
        student_id = os.path.splitext(image_name)[0]
        student_ids.extend([student_id] * len(encodings))
    
    print(f"Encodings extracted for {len(encode_list)} faces")
    
    with open(encoding_file, 'wb') as file:
        pickle.dump([encode_list, student_ids], file)
    print("Encodings saved.")

# Run the encoding function
encode_images(images_folder)

Processing image: img\12345.png
Processing image: img\20660.png
Processing image: img\321653.png
Processing image: img\43562.png
Processing image: img\452749.png
Processing image: img\456214.png
Processing image: img\54362.png
Processing image: img\70001.png
Processing image: img\76543.png
Processing image: img\852741.png
Processing image: img\963852.png
Encodings extracted for 11 faces
Encodings saved.


# Detector

In [1]:
# Import necessary libraries
import os
import pickle
import numpy as np
import cv2
import face_recognition
import cvzone
import firebase_admin
from firebase_admin import credentials, db, storage
from datetime import datetime

# Initialize Firebase
cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': "https://facial-recognition-623bd-default-rtdb.firebaseio.com/",
    'storageBucket': "facial-recognition-623bd.appspot.com"  
})

bucket = storage.bucket()

# Setup webcam
cap = cv2.VideoCapture(0)  # Ensure correct webcam index
cap.set(3, 640)
cap.set(4, 480)

# Load background image
imgBackground = cv2.imread('Resources/background.png')

# Import mode images
folderModePath = 'Resources/Modes'
modePathList = [f for f in os.listdir(folderModePath) if os.path.isfile(os.path.join(folderModePath, f))]
imgModeList = []
for path in modePathList:
    img = cv2.imread(os.path.join(folderModePath, path))
    if img is not None:
        imgModeList.append(img)
    else:
        print(f"Failed to load image: {path}")
        imgModeList.append(cv2.imread('Resources/Modes/default.png'))  # Add default image if loading fails

# Load the encoding file
print("Loading Encode File ...")
with open('EncodeFiles.p', 'rb') as file:
    encodeListKnownWithIds = pickle.load(file)
encodeListKnown, studentIds = encodeListKnownWithIds
print("Encode File Loaded")

# Path to default image for unknown faces
default_image_path = 'Resources/Modes/default.png'  # Ensure this path is correct
default_image = cv2.imread(default_image_path)

modeType = 0
counter = 0
id = -1
imgStudent = []

# Set a threshold for the face distance
face_distance_threshold = 0.6  # Adjust this value based on your requirement

while True:
    success, img = cap.read()
    if not success:
        print("Failed to capture image")
        break
    
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)

    faceCurFrame = face_recognition.face_locations(imgS)
    encodeCurFrame = face_recognition.face_encodings(imgS, faceCurFrame)

    imgBackground[165:165 + 480, 70:70 + 640] = img

    # Ensure modeType is within bounds
    if 0 <= modeType < len(imgModeList) and imgModeList[modeType] is not None:
        imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
    else:
        # Use default image for out-of-bounds modeType
        imgBackground[44:44 + 633, 808:808 + 414] = default_image

    if faceCurFrame:
        for encodeFace, faceLoc in zip(encodeCurFrame, faceCurFrame):
            matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
            faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)

            matchIndex = np.argmin(faceDis)
            if matches[matchIndex] and faceDis[matchIndex] < face_distance_threshold:
                y1, x2, y2, x1 = [coord * 4 for coord in faceLoc]
                bbox = 55 + x1, 162 + y1, x2 - x1, y2 - y1
                imgBackground = cvzone.cornerRect(imgBackground, bbox, rt=0)
                id = studentIds[matchIndex]
                if counter == 0:
                    cvzone.putTextRect(imgBackground, "Loading", (275, 400))
                    cv2.imshow("Face Attendance", imgBackground)
                    cv2.waitKey(1)
                    counter = 1
                    modeType = 1
            else:
                modeType = 4  # Set mode for unrecognized face
                if 0 <= modeType < len(imgModeList) and imgModeList[modeType] is not None:
                    imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
                else:
                    imgBackground[44:44 + 633, 808:808 + 414] = default_image
                cvzone.putTextRect(imgBackground, "Face Not Recognizable", (275, 400))

        if counter != 0:
            if counter == 1:
                studentInfo = db.reference(f'Employees/{id}').get()
                if studentInfo:
                    blob = bucket.get_blob(f'img/{id}.png')
                    if blob:
                        array = np.frombuffer(blob.download_as_string(), np.uint8)
                        imgStudent = cv2.imdecode(array, cv2.IMREAD_COLOR)
                    
                    lastAttendanceTime = studentInfo.get('last_attendance_time')
                    if lastAttendanceTime:
                        datetimeObject = datetime.strptime(lastAttendanceTime, "%Y-%m-%d %H:%M:%S")
                        secondsElapsed = (datetime.now() - datetimeObject).total_seconds()
                        if secondsElapsed > 30:
                            ref = db.reference(f'Employees/{id}')
                            studentInfo['total_attendance'] += 1
                            ref.child('total_attendance').set(studentInfo['total_attendance'])
                            ref.child('last_attendance_time').set(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
                        else:
                            modeType = 3
                            counter = 0
                            if 0 <= modeType < len(imgModeList) and imgModeList[modeType] is not None:
                                imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
                            else:
                                imgBackground[44:44 + 633, 808:808 + 414] = default_image
                    else:
                        print("No last attendance time found for Employees ID:", id)
                else:
                    print("No Employee info found for Employees ID:", id)

            if modeType != 3 and modeType != 4:
                if 10 < counter < 20:
                    modeType = 2

                if 0 <= modeType < len(imgModeList) and imgModeList[modeType] is not None:
                    imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]

                if counter <= 10 and studentInfo:
                    cv2.putText(imgBackground, str(studentInfo['total_attendance']), (861, 125),
                                cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 1)
                    cv2.putText(imgBackground, str(studentInfo['major']), (1006, 550),
                                cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)
                    cv2.putText(imgBackground, str(id), (1006, 493),
                                cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 255, 255), 1)

                    (w, h), _ = cv2.getTextSize(studentInfo['name'], cv2.FONT_HERSHEY_COMPLEX, 1, 1)
                    offset = (414 - w) // 2
                    cv2.putText(imgBackground, str(studentInfo['name']), (808 + offset, 445),
                                cv2.FONT_HERSHEY_COMPLEX, 1, (50, 50, 50), 1)

                    if imgStudent.any():
                        imgBackground[175:175 + 216, 909:909 + 216] = imgStudent
                    else:
                        imgBackground[175:175 + 216, 909:909 + 216] = np.zeros((216, 216, 3), dtype=np.uint8)
        
                counter += 1
                if counter >= 20:
                    counter = 0
                    modeType = 0
                    studentInfo = {}
                    imgStudent = []
                    if 0 <= modeType < len(imgModeList) and imgModeList[modeType] is not None:
                        imgBackground[44:44 + 633, 808:808 + 414] = imgModeList[modeType]
                    else:
                        imgBackground[44:44 + 633, 808:808 + 414] = default_image
    else:
        modeType = 0  
        counter = 0

    cv2.imshow("Face Attendance", imgBackground)
    if cv2.waitKey(1) & 0xFF == ord('y'):
        break

cap.release()
cv2.destroyAllWindows()


Loading Encode File ...
Encode File Loaded
No last attendance time found for Employees ID: 43562
No Employee info found for Employees ID: 54362
No Employee info found for Employees ID: 54362
No Employee info found for Employees ID: 54362
No Employee info found for Employees ID: 54362
No Employee info found for Employees ID: 54362
No Employee info found for Employees ID: 54362
No Employee info found for Employees ID: 54362
No Employee info found for Employees ID: 54362
No Employee info found for Employees ID: 54362
No Employee info found for Employees ID: 54362
No Employee info found for Employees ID: 54362
No Employee info found for Employees ID: 54362
No Employee info found for Employees ID: 54362
No Employee info found for Employees ID: 54362
No Employee info found for Employees ID: 54362
No Employee info found for Employees ID: 54362
No last attendance time found for Employees ID: 321653
No last attendance time found for Employees ID: 321653
No last attendance time found for Employe

# Storing Data into Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

cred = credentials.Certificate("serviceAccountKey.json")
firebase_admin.initialize_app(cred, {
    'databaseURL': "https://facial-recognition-623bd-default-rtdb.firebaseio.com/"
})

ref = db.reference('Employees')

data = {
    "321653":
        {
            "name": "Rishbah Jain",
            "major": "Robotics",
            "total_attendance": 7,
        },
    "20660":
        {
            "name": "Seo ye ji",
            "major": "Actress",
            "total_attendance": 7,
        },
    "70001":
        {
            "name": "Reyna",
            "major": "Actress",
            "total_attendance": 7,
        },
    "452749":
        {
            "name": "Mr. Will",
            "major": "Actress",
            "total_attendance": 7,
        },
    "76543":
        {
            "name": "sanjeev",
            "major": "Robotics",
            "total_attendance": 1,
        }, 
    "12345":
        {
            "name": "saught",
            "major": "Robotics",
            "total_attendance": 2,
        },    
    "43562":
        {
            "name": "Nitin Rawat",
            "major": "Machine Larning",
            "total_attendance": 7,
        },   
               
    "852741":
        {
            "name": "Emly Blunt",
            "major": "Economics",
            "total_attendance": 12,
        },
    "963852":
        {
            "name": "Elon Musk",
            "major": "Physics",  
            "total_attendance": 7,
        }
}

for key, value in data.items():
    ref.child(key).set(value)